In [1]:

from skimage.measure import label

#Scientific computing 
import numpy as np
import os
#Pytorch packages
import torch
from torch import nn
import torch.optim as optim
import torchvision
#from torchvision import datasets
from tensorboardX import SummaryWriter
#Visulization
import matplotlib.pyplot as plt
from torchvision import transforms
from PIL import Image
#Others
#from torch.utils.data import DataLoader, Subset
from torch.autograd import Variable
import matplotlib.pyplot as plt
import copy
#from finetuneSAM.utils.dataset import Public_dataset
import torch.nn.functional as F
from torch.nn.functional import one_hot
from pathlib import Path
from tqdm import tqdm
#from finetuneSAM.utils.losses import DiceLoss
#from finetuneSAM.utils.dsc import dice_coeff_multi_class
import cv2
import monai
#from finetuneSAM.utils.utils import vis_image
#import cfg
import json



# General
import numpy as np
import torch


from skimage.measure import label

#Scientific computing 
import numpy as np
import os
#Pytorch packages
import torch

import torchvision


import matplotlib.pyplot as plt

from PIL import Image


import matplotlib.pyplot as plt
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


device(type='cuda')

In [2]:
#Fine-Tune Sam
from finetuneSAM.models.sam import SamAutomaticMaskGenerator, SamPredictor, sam_model_registry
from finetuneSAM.cfg import parse_args
args =  parse_args()


# setting if_mask_decoder_adapter = True puts adapters inside 2-way transformer blocks
# this does not change the number of decoder 2-way transformer blocks (def = 2)
# decoder_adapt_depth denotes how many of the two 2-way transformer blocks are adapted


# setting if_encoder_adapter = True puts adapters inside TinyViTBlocks in the encoder
# this does not change the number of encoder TinyViTBlocks (def = 4)
# encoder_adapt_depth (e.g. [1,2]) denotes how deep blocks will be adapted

args.finetune_type = "vanilla"
#args.finetune_type = "adapter"# "vanilla"
#args.if_mask_decoder_adapter = True
#args.image_size = 512
#args.decoder_adapt_depth = 1
args.num_cls = 3


# Load the pre-trained model
sam_checkpoint = "mobile_sam.pt"
# Define the model type: Tiny Vit
model_type = "vit_t"

# Load the model
mobile_sam_f = sam_model_registry[model_type](args , checkpoint=sam_checkpoint)
# Move the model to the device
mobile_sam_f = mobile_sam_f.to(device=device)
# Set the model to evaluation mode
#mobile_sam_f.eval()

predictor_f = SamPredictor(mobile_sam_f)

c:\Users\newma\AppData\Local\Programs\Python\Python310\lib\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
c:\Users\newma\AppData\Local\Programs\Python\Python310\lib\site-packages\timm\models\registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
c:\KU\ATIA\finetuneSAM\models\sam\modeling\tiny_vit_sam.py:760: UserWarning: Overwriting tiny_vit_5m_224 in registry with finetuneSAM.models.sam.modeling.tiny_vit_sam.tiny_vit_5m_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  return register_model(fn_wrapper)
c:\KU\ATIA\finetuneSAM\models\sam\modeling\tiny_vit_sam.p

In [3]:


from finetuneSAM.models.sam.utils.transforms import ResizeLongestSide

def preprocess(model, input_img_size, image: np.ndarray, point = None):
    torch.no_grad()
    org_shape = image.shape

    transform = ResizeLongestSide(input_img_size) # can be changed?

    #set_image
    input_image = transform.apply_image(image)
    input_image_torch = torch.as_tensor(input_image, device=device)
    input_image_torch = input_image_torch.permute(2, 0, 1).contiguous()[None, :, :, :]

    #set_torch_image
    transformed_image = input_image_torch
    original_image_size = input_image.shape[:2]
    input_size = tuple(transformed_image.shape[-2:])
    transformed_image = model.preprocess(transformed_image)
    #input_image = torch.tensor(transformed_image*1.0/255) 
    features = model.image_encoder(transformed_image)


    coords_torch, labels_torch, box_torch, mask_input_torch = None, None, None, None
    # masks, iou_predictions, low_res_masks = self.predict_torch(
    #     coords_torch,
    #     labels_torch,
    #     box_torch,
    #     mask_input_torch,
    #     multimask_output,
    #     return_logits=return_logits,
    # )

    point_coords = transform.apply_coords(np.array([[940, 730]]).reshape(1,2), org_shape[:2])
    coords_torch = torch.as_tensor(point_coords, dtype=torch.float, device=device)
    labels_torch = torch.as_tensor(np.array([1]), dtype=torch.int, device=device)
    coords_torch, labels_torch = coords_torch[None, :, :], labels_torch[None, :]


    points=(coords_torch, labels_torch)
    sparse_embeddings, dense_embeddings = model.prompt_encoder(
            points=points,
            boxes=box_torch,
            masks=mask_input_torch,
        )

    # Predict masks
    low_res_masks, iou_predictions = model.mask_decoder(
        image_embeddings = features,
        image_pe= model.prompt_encoder.get_dense_pe(),
        sparse_prompt_embeddings=sparse_embeddings,
        dense_prompt_embeddings=dense_embeddings,
        multimask_output= True,
    )

    # Upscale the masks to the original image resolution
    masks = model.postprocess_masks(low_res_masks, input_size, org_shape[:2])# original_image_size)
    
    #masks = masks > 0.0

    # if not [0] then batched ?
    return masks[0].detach().cpu().numpy(), iou_predictions[0].detach().cpu().numpy(), low_res_masks[0].detach().cpu().numpy()

In [12]:
image = Image.open('picture4.jpg')
print(image.size)
image = np.asarray(image)
#image = image[0:254,0:254]
#image = image[0:1024,0:1024]
image.shape

(2048, 1365)


(1365, 2048, 3)

In [13]:
predictor_f.set_image(image)

input identical True
input_image_torch(after cast):  True
input_image_torch (after permute)  True
input_image after preproc:  True
features identical True


In [6]:
def format_results(masks, scores, logits, filter=0):
    annotations = []
    n = len(scores)
    for i in range(n):
        annotation = {}

        mask = masks[i]
        tmp = np.where(mask != 0)
        if np.sum(mask) < filter:
            continue
        annotation["id"] = i
        annotation["segmentation"] = mask
        annotation["bbox"] = [
            np.min(tmp[0]),
            np.min(tmp[1]),
            np.max(tmp[1]),
            np.max(tmp[0]),
        ]
        annotation["score"] = scores[i]
        annotation["area"] = annotation["segmentation"].sum()
        annotations.append(annotation)
    return annotations

In [14]:
masks, scores, logits = preprocess(mobile_sam_f, 1024, image)
print(masks.shape)
print(scores)
#results = format_results(masks, scores, logits, filter=0)
#results[0]

(3, 1365, 2048)
[0.85441417 1.0176673  1.0066909 ]


In [8]:
def put_massk(image,mask):
    img = image.copy()
    for i in range(mask.shape[0]):
        for j in range(mask.shape[1]):
            if mask[i,j] == 1:
                img[i,j] = [255,0,0]
    return img

In [10]:
r = put_massk(image,masks[1])
Image.fromarray(r).show()

In [18]:

m1 = logits[2]
m = 1* m1
m = m*255

#set rgb to 255 for all 1s in the mask

Image.fromarray(m).show()

In [16]:
#Image.fromarray(results[0]["segmentation"]).show()
#Image.fromarray(results[0]["segmentation"]).show()

m1 = masks[1]
m = 1* m1
m = m*255

#set rgb to 255 for all 1s in the mask

Image.fromarray(m).show()


In [31]:
predictor_f.set_image(image)
FEAT_AUTO = predictor_f.features
#masks, scores, logits = predictor_f.predict(None,None)#, multimask_output=True)

p = np.array([940,730]).reshape(1,2)
masks, scores, logits = predictor_f.predict(p,point_labels= np.array([1]))
# p,point_labels= np.array([1])
scores

input identical True
input_image_torch(after cast):  True
input_image_torch (after permute)  True
input_image after preproc:  True
features identical True


array([0.84140253, 0.9449352 , 0.8273449 ], dtype=float32)

In [25]:
q = put_massk(image,masks[2])
Image.fromarray(q).show()

In [36]:
def dice_coeff_multi_class(pred, target, n_classes):
    """Calculate the mean Dice Coefficient for multi-class data."""
    dice_scores = []
    for cls in range(n_classes):  # Iterate over each class
        pred_cls = (pred == cls).long()  # Binary mask for current predicted class
        target_cls = (target == cls).long()  # Binary mask for current actual class

        smooth = 1.0
        intersection = (pred_cls & target_cls).float().sum((1, 2))  # Sum over height and width dimensions
        union = pred_cls.float().sum((1, 2)) + target_cls.float().sum((1, 2))
        
        dice = (2. * intersection + smooth) / (union + smooth)
        dice_scores.append(dice)
    
    return torch.stack(dice_scores).mean() 

In [29]:


sam = mobile_sam_f

data = {'image':train_imgs, 'mask':train_masks}


if args.finetune_type == 'adapter':
    for n, value in sam.named_parameters():
        if "Adapter" not in n: # only update parameters in adapter
            value.requires_grad = False
    print('if update encoder:',args.if_update_encoder)
    print('if image encoder adapter:',args.if_encoder_adapter)
    print('if mask decoder adapter:',args.if_mask_decoder_adapter)
    if args.if_encoder_adapter:
        print('added adapter layers:',args.encoder_adapter_depths)
    
# elif args.finetune_type == 'vanilla' and args.if_update_encoder==False:   
#     print('if update encoder:',args.if_update_encoder)
#     for n, value in sam.image_encoder.named_parameters():
#         value.requires_grad = False
# elif args.finetune_type == 'lora':
#     print('if update encoder:',args.if_update_encoder)
#     print('if image encoder lora:',args.if_encoder_lora_layer)
#     print('if mask decoder lora:',args.if_decoder_lora_layer)
#     sam = LoRA_Sam(args,sam,r=4).sam
# sam.to('cuda')
b_lr = args.lr
epochs = 80
    
optimizer = optim.AdamW(sam.parameters(), lr=b_lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.1, amsgrad=False)
optimizer.zero_grad()
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5) #learning rate decay
criterion1 = monai.losses.DiceLoss(sigmoid=True, squared_pred=True, to_onehot_y=True,reduction='mean')
criterion2 = nn.CrossEntropyLoss()
iter_num = 0
max_iterations = epochs * train_points
writer = SummaryWriter(checkpoints_path + '/log')

pbar = tqdm(range(epochs))
val_largest_dsc = 0
last_update_epoch = 0
for epoch in pbar:


    sam.train()
    train_loss = 0
    # batches here?
    #for i,_ in enumerate(tqdm(data['image'])):
    for i in range(1):
        imgs = data['image'][0:1].cuda()
        msks = torchvision.transforms.Resize((args.out_size,args.out_size))(data['mask'][0:1])
        msks = msks.cuda()



        if args.if_update_encoder:
            img_emb = sam.image_encoder(imgs)
        else:
            with torch.no_grad():
                img_emb = sam.image_encoder(imgs)
        
        # get default embeddings
        sparse_emb, dense_emb = sam.prompt_encoder(
            points=None,
            boxes=None,
            masks=None,
        )
        pred, _ = sam.mask_decoder(
                        image_embeddings=img_emb,
                        image_pe=sam.prompt_encoder.get_dense_pe(), 
                        sparse_prompt_embeddings=sparse_emb,
                        dense_prompt_embeddings=dense_emb, 
                        #multimask_output=True,
                        multimask_output=False,
                        )
        
        print("THIS WORKS")
        print(msks.shape)
        print(msks.float().shape)
        print(pred.shape)
        


        #loss_dice = criterion1(pred,msks.float()) 
        loss_ce = criterion2(pred,torch.squeeze(msks.long(),1))
        loss =  loss_ce
        #loss =  loss_dice + loss_ce
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad(set_to_none=True)
        
        if args.if_warmup and iter_num < args.warmup_period:
            lr_ = args.lr * ((iter_num + 1) / args.warmup_period)
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr_

        else:
            if args.if_warmup:
                shift_iter = iter_num - args.warmup_period
                assert shift_iter >= 0, f'Shift iter is {shift_iter}, smaller than zero'
                lr_ = args.lr * (1.0 - shift_iter / max_iterations) ** 0.9  # learning rate adjustment depends on the max iterations
                for param_group in optimizer.param_groups:
                    param_group['lr'] = lr_
            else:
                lr_ = args.lr

        train_loss += loss.item()
        iter_num+=1
        writer.add_scalar('info/lr', lr_, iter_num)
        writer.add_scalar('info/total_loss', loss, iter_num)
        writer.add_scalar('info/loss_ce', loss_ce, iter_num)
        writer.add_scalar('info/loss_dice', loss_dice, iter_num)

    train_loss /= (i+1)
    pbar.set_description('Epoch num {}| train loss {} \n'.format(epoch,train_loss))

    # if epoch%2==0:
    #     eval_loss=0
    #     dsc = 0
    #     sam.eval()
    #     with torch.no_grad():
    #         for i in tqdm(data['image']):
    #             imgs = data['image'][i].cuda()
    #             msks = torchvision.transforms.Resize((args.out_size,args.out_size))(data['mask'][i])
    #             msks = msks.cuda()

    #             img_emb= sam.image_encoder(imgs)
    #             sparse_emb, dense_emb = sam.prompt_encoder(
    #                 points=None,
    #                 boxes=None,
    #                 masks=None,
    #             )
    #             pred, _ = sam.mask_decoder(
    #                             image_embeddings=img_emb,
    #                             image_pe=sam.prompt_encoder.get_dense_pe(), 
    #                             sparse_prompt_embeddings=sparse_emb,
    #                             dense_prompt_embeddings=dense_emb, 
    #                             multimask_output=True,
    #                             )
    #             loss = criterion1(pred,msks.float()) + criterion2(pred,torch.squeeze(msks.long(),1))
    #             eval_loss +=loss.item()
    #             dsc_batch = dice_coeff_multi_class(pred.argmax(dim=1).cpu(), torch.squeeze(msks.long(),1).cpu().long(),args.num_cls)
    #             dsc+=dsc_batch

    #         eval_loss /= (i+1)
    #         dsc /= (i+1)
            
    #         writer.add_scalar('eval/loss', eval_loss, epoch)
    #         writer.add_scalar('eval/dice', dsc, epoch)
            
    #         print('Eval Epoch num {} | val loss {} | dsc {} \n'.format(epoch,eval_loss,dsc))
    #         if dsc>val_largest_dsc:
    #             val_largest_dsc = dsc
    #             last_update_epoch = epoch
    #             print('largest DSC now: {}'.format(dsc))
    #             torch.save(sam.state_dict(),checkpoints_path + '/checkpoint_best.pth')
    #         elif (epoch-last_update_epoch)>20:
    #             # the network haven't been updated for 20 epochs
    #             print('Training finished###########')
    #             break
writer.close()

if update encoder: False
if image encoder adapter: False
if mask decoder adapter: True


  0%|          | 0/80 [00:00<?, ?it/s]


RuntimeError: Given groups=1, weight of size [32, 3, 3, 3], expected input[1, 1024, 1024, 3] to have 3 channels, but got 1024 channels instead